<a href="https://colab.research.google.com/github/PyBlin/Study/blob/main/PyML/Chapter4_Classification/Chap4_3_Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.Ensemble Learning

* Voting
>* 서로 다른 알고리즘을 가진 분류기를 결합합니다.
>* ex) 하나의 데이터셋을 각각 LR, k-NN, SVM을 사용하여 Voting 합니다.

* Bagging
>* 각각의 분류기가 모두 같은 유형의 알고리즘 기반이지만, 데이터 샘플링이 다릅니다.
>* 대표적으로 Random Forest 알고리즘이 있습니다.
>* 이렇게 개별 분류기에게 원본 데이터를 샘플링해서 추출하는 방식을 "Bootstrapping 분할 방식"이라고 합니다.
>* "Bootstrapping" 방식으로 샘플링된 데이터셋에 대해서 학습하고 예측한 결과를 "Voting"을 통해서 최종 예측 결과를 선정하는 방식을 "Bagging Ensemble"입니다.
>* 교차 검증과 다르게, Bagging은 데이터셋간에 중첩을 허용합니다.
>* 즉, 1만개의 데이터를 10개의 분류기가 Bagging 하더라도, 각 1천개의 데이터 내에는 중복된 데이터가 있습니다.
>* ex) 여러 개의 데이터셋을 모두 DT를 사용하여 Bagging 합니다.

* Boosting
>* 여러 개의 같은 분류기가 순차적으로 학습을 수행하되, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측할 수 있도록 다음 분류기에게 가중치( weight )를 부여합니다.
>* 계속해서 가중치를 부여해서 Boosting이라고 합니다.
>* 예측 성능이 뛰어나 Ensemble 학습을 주도하고 있습니다.
>* 대표적으로 Gradient Boost, XGBoost, LightGBM 등이 있습니다.

* Stacking
>* 여러 모델의 예측 결과값을 다시 학습 데이터로 만들어서 다른 모델( meta model)로 재학습시키는 방식입니다.

## 3.1 Hard Voting & Soft Voting

* Hard Voting
>* 다수결과 비슷합니다.
>* 예측한 결과값 중 다수의 분류기가 결정한 예측값을 최종으로 선정합니다.

* Soft Voting
>* 분류기들의 레이블 결정값을 모두 더하고 평균 중에서 확률이 가장 높은 레이블 값을 최종으로 선정합니다.
>* 일반적으로 많이 적용됩니다.

## 3.2 Voting Classifier

* 사이킷런은 Voting Ensemble을 구현한 `VotingClassifier` 클래스를 제공합니다.
* 위스콘신 유방암 데이터셋을 예측해 봅시다!
* 악성/양성 종양 여부를 결정하는 이진 분류 데이터셋이며, 종양의 크기, 모양 등의 형태와 관련한 많은 피처가 있습니다.
* Logistic Regression, KNN 기반으로 수행해 봅시다!

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [2]:
# 개별 모델은 Logistic Regressin, KNN
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 soft voting 기반의 ensemble 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], 
                          voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2, 
                                                    random_state=156)

# Voting Classifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print(f"Voting Classifier Accuracy : {accuracy_score(y_test, pred):.4f}")

# 개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print(f"{class_name} Accuracy : {accuracy_score(y_test, pred):.4f}")

Voting Classifier Accuracy : 0.9474
LogisticRegression Accuracy : 0.9386
KNeighborsClassifier Accuracy : 0.9386


* Voting 분류기가 조금 높은데, 무조건 향상되는건 아닙니다. 오히려 기반 분류기가 가장 성능이 좋을 수도 있습니다.
* 그럼에도 불구하고 단일 ML 알고리즘보다 뛰어납니다.
* 고정된 데이터셋에서 단일 ML 알고리즘이 뛰어날지라도, 현실에서는 어렵습니다.
* 현실에서도 똑똑한 사람들로만 구성된 팀보다는 다양한 경험의 백그라운드를 가진 사람들로 구성된 팀이 더 성공할 가능성이 높습니다.
* 높은 유연성은 해결책을 제시할 수 있습니다.
* 이런 관점에서 편향-분산 Trade-off는 ML 모델이 극복해야 할 중요 과제입니다.